<a href="https://colab.research.google.com/github/dkumar-23/Masters_ML/blob/main/Assgn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>